In [1]:
import pandas as pd
import numpy as np

In [58]:
train=pd.read_csv('Train.csv')
test=pd.read_csv('Test.csv')

In [59]:
train.head()

,City,Location_Score,Internal_Audit_Score,External_Audit_Score,Fin_Score,Loss_score,Past_Results,IsUnderRisk
0,2,8.032,14,8,3,6,0,1
1,31,77.730,8,3,3,8,1,0
2,40,59.203,3,12,11,3,0,1
3,12,73.080,4,5,7,6,0,0
4,4,15.666,13,15,6,7,2,1


In [60]:
test.head()

,City,Location_Score,Internal_Audit_Score,External_Audit_Score,Fin_Score,Loss_score,Past_Results
0,41,18.272,13,12,9,7,0
1,17,64.799,6,10,7,4,1
2,31,68.890,3,3,7,8,0
3,3,16.492,15,10,7,4,1
4,9,17.178,7,3,8,3,1


In [61]:
train.describe()

,City,Location_Score,Internal_Audit_Score,External_Audit_Score,Fin_Score,Loss_score,Past_Results,IsUnderRisk
count,543.000000,543.000000,543.000000,543.000000,543.000000,543.000000,543.000000,543.000000
mean,19.576427,32.259346,8.189687,7.327808,7.046041,5.530387,0.613260,0.626151
std,14.722687,24.887291,3.312022,3.450667,3.156978,1.839124,0.764509,0.484270
min,0.000000,5.185000,3.000000,3.000000,3.000000,3.000000,0.000000,0.000000
25%,6.000000,13.565000,6.000000,5.000000,5.000000,4.000000,0.000000,0.000000
50%,14.000000,19.803000,8.000000,7.000000,7.000000,6.000000,1.000000,1.000000
75%,37.000000,62.594000,11.000000,10.000000,8.000000,7.000000,1.000000,1.000000
max,44.000000,80.809000,15.000000,15.000000,15.000000,13.000000,10.000000,1.000000


In [62]:
test.isnull().sum()

City                    0
Location_Score          0
Internal_Audit_Score    0
External_Audit_Score    0
Fin_Score               0
Loss_score              0
Past_Results            0
dtype: int64

In [63]:
train['IsUnderRisk'].value_counts()

1    340
0    203
Name: IsUnderRisk, dtype: int64

In [64]:
from sklearn import preprocessing
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.IsUnderRisk.values)

In [65]:
train.drop('IsUnderRisk',axis=1,inplace=True)

In [67]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
train=sc.fit_transform(train)
test=sc.fit_transform(test)

In [68]:
train

array([[-1.19493362, -0.97438028,  1.75592772, ..., -1.28279992,
         0.2555816 , -0.80290162],
       [ 0.77663173,  1.82874789, -0.05732506, ..., -1.28279992,
         1.34405853,  0.50633436],
       [ 1.38849684,  1.08362528, -1.56836904, ...,  1.25360556,
        -1.37713379, -0.80290162],
       ...,
       [-1.26291864,  1.88959803, -0.35953385, ..., -1.28279992,
         1.34405853,  0.50633436],
       [ 1.38849684,  1.48327355, -0.35953385, ..., -0.96574923,
        -0.28865686,  0.50633436],
       [-0.44709849, -0.3590418 ,  1.75592772, ...,  0.93655488,
        -1.37713379,  1.81557034]])

In [69]:
from sklearn.model_selection import train_test_split

In [70]:
x_train,x_cv,y_train,y_cv=train_test_split(train,y,test_size=0.10,random_state=1234)

In [71]:
x_train

array([[-1.12694861, -0.99115127,  1.45371893, ..., -1.28279992,
         1.34405853,  0.50633436],
       [ 0.23275164,  1.60618037,  0.24488374, ..., -0.96574923,
         0.79982006, -0.80290162],
       [-0.44709849, -0.81008893,  0.84930133, ...,  1.25360556,
         0.79982006,  0.50633436],
       ...,
       [ 1.18454181, -0.36495388, -0.66174265, ..., -0.64869855,
        -0.28865686,  0.50633436],
       [ 1.38849684, -0.61273822,  1.15151013, ..., -0.01459718,
        -0.28865686,  0.50633436],
       [ 0.77663173, -0.97566726, -0.96395145, ..., -0.01459718,
        -1.37713379, -0.80290162]])

In [72]:
from sklearn.linear_model import LogisticRegression
lc=LogisticRegression()
lc.fit(x_train,y_train)
lc_cv_pred=lc.predict_proba(x_cv)

In [73]:
from sklearn.metrics import log_loss

In [125]:
print(log_loss(y_cv,lc_cv_pred))

0.34588305241180123


In [126]:
lc_pred=lc.predict_proba(test)

In [108]:
from lightgbm import LGBMClassifier
lgb=LGBMClassifier(boosting_type='gbdt',max_depth=10)
lgb.fit(x_train,y_train)
lgb_cv_pred=lgb.predict_proba(x_cv)

In [109]:
print(log_loss(y_cv,lgb_cv_pred))

0.32319608687800466


In [110]:
lgb_pred=lgb.predict_proba(test)

In [111]:
sample=pd.DataFrame(lgb_pred)

In [112]:
sample.head()

,0,1
0,0.000108,0.999892
1,0.010539,0.989461
2,0.950509,0.049491
3,0.000076,0.999924
4,0.129175,0.870825


In [99]:
sample.to_excel('sample_lgb.xlsx',index=False)